In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization

import cv2
from tqdm import tqdm

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

Using TensorFlow backend.
/home/chicm/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']
thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]

labels_old= ['haze', 'cultivation', 'blooming', 'partly_cloudy', 'habitation', 'primary', 
            'road', 'agriculture', 'selective_logging', 'artisinal_mine', 'slash_burn', 
            'blow_down', 'cloudy', 'bare_ground', 'conventional_mine', 'clear', 'water']
thres2=[]
for tag in labels_old:
    for i, item in enumerate(labels):
        if item == tag:
            thres2.append(thres[i])
print(thres2)

label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

DATA_DIR = '/home/chicm/data/planet'
RESULT_DIR = DATA_DIR+'/results'

[0.24, 0.23, 0.04, 0.14, 0.19, 0.22, 0.24, 0.25, 0.12, 0.33, 0.1, 0.07, 0.16, 0.17, 0.2, 0.23, 0.26]


In [10]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [11]:
from tqdm import tqdm

result = np.array(load_array(RESULT_DIR+'/preds'))
result = pd.DataFrame(result, columns = labels_old)


#thres = [0.07, 0.17, 0.2, 0.04, 0.23, 0.33, 0.24, 0.22, 0.1, 0.19, 0.23, 0.24, 0.12, 0.14, 0.25, 0.26, 0.16]
preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > thres2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

print(result)

100%|██████████| 40669/40669 [01:09<00:00, 584.76it/s]

           haze  cultivation  blooming  partly_cloudy  habitation   primary  \
0      0.010413     0.061171  0.018753       0.014981    0.008437  0.994475   
1      0.001926     0.071439  0.028720       0.059257    0.005545  0.996499   
2      0.001487     0.010154  0.001521       0.999957    0.002562  0.996174   
3      0.002417     0.098846  0.029723       0.036244    0.008002  0.997856   
4      0.002046     0.010045  0.001570       0.871230    0.006314  0.953009   
5      0.002041     0.037091  0.011318       0.012539    0.005405  0.997593   
6      0.052255     0.197007  0.003246       0.453348    0.697254  0.993224   
7      0.030247     0.015061  0.002288       0.006936    0.992854  0.788347   
8      0.003832     0.037835  0.003167       0.013642    0.009155  0.996440   
9      0.855388     0.088299  0.001699       0.027752    0.016079  0.988223   
10     0.001726     0.019529  0.001825       0.992512    0.004926  0.993394   
11     0.002561     0.979194  0.003909       0.00465

In [12]:
df_test = pd.read_csv(DATA_DIR+'/sample_submission.csv')
df_test['tags'] = preds
df_test

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,partly_cloudy primary
3,test_3,primary clear
4,test_4,partly_cloudy primary cloudy
5,test_5,primary clear
6,test_6,partly_cloudy habitation primary road agricult...
7,test_7,habitation primary road agriculture clear
8,test_8,primary clear
9,test_9,haze primary agriculture


In [13]:
df_test.to_csv(RESULT_DIR+'/sub12.csv', index=False)